In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Since lyrics were auto generated using whisper's medium model, some lyrics are garbage. this code got rid of those


In [ ]:
LYRICS_CSV = '/content/drive/MyDrive/DL-Project/data/lyrics/lyrics.csv'
AUDIO_DIR = '/content/drive/MyDrive/DL-Project/data/audio'
SAVE_DIR = '/content/drive/MyDrive/DL-Project/data/processed_medium'
CSV_PATH = '/content/drive/MyDrive/DL-Project/data/lyrics/curated_lyrics.csv'
os.makedirs(SAVE_DIR, exist_ok=True)

# loading csv
og = pd.read_csv(LYRICS_CSV)
df = og.copy()
original_count = len(df)

# Ensure track_id is int
df['track_id'] = pd.to_numeric(df['track_id'], errors='coerce').fillna(-1).astype(int)

# filtering garbage lyrucs
garbage_phrases = ["thanks for watching", "thanks for watching!", "thank you for watching!", "thank you for watching!" "subscribe", "music", "applause", "music playing", "music you", "copyright", "generated by"]

def is_valid_lyric(text):
    if not isinstance(text, str): return False
    if len(text) < 40: return False # too short for lyric
    text_lower = text.lower()
    if any(phrase in text_lower for phrase in garbage_phrases): return False
    return True

df = df[df['lyrics'].apply(is_valid_lyric)]
print(f"Cleaned Lyrics: {original_count} -> {len(df)} songs")

#intersecting lyrics + ids
available_audio = set([int(f.split('.')[0]) for f in os.listdir(AUDIO_DIR) if f.endswith('.wav') and f.split('.')[0].isdigit()])
df = df[df['track_id'].isin(available_audio)]
print(f"Final Gold Set (Audio + Valid Lyrics): {len(df)} songs")

# Sort by ID to ensure alignment
df = df.sort_values('track_id').reset_index(drop=True)

##df.to_csv(CSV_PATH, index=False)

Cleaned Lyrics: 7997 -> 3216 songs
Final Gold Set (Audio + Valid Lyrics): 3216 songs


In [ ]:
# text embedding using MPNET base 2 model
print("Loading all-mpnet-base-v2")
model = SentenceTransformer('all-mpnet-base-v2')
text_embeddings = model.encode(df['lyrics'].tolist(), show_progress_bar=True, batch_size=32)

np.save(os.path.join(SAVE_DIR, 'text_embeddings.npy'), text_embeddings)
np.save(os.path.join(SAVE_DIR, 'curated_lyrics_track_ids.npy'), df['track_id'].values)
print(f"Done. Text Embeddings shape: {text_embeddings.shape}")

Loading all-mpnet-base-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Done. Text Embeddings shape: (3216, 768)


In [ ]:
import librosa
import numpy as np
import pandas as pd
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#file paths
AUDIO_DIR = '/content/drive/MyDrive/DL-Project/data/audio'
SAVE_DIR = '/content/drive/MyDrive/DL-Project/data/processed_medium'
META_PATH = '/content/drive/MyDrive/DL-Project/metadata.csv'
RESULTS_CSV_PATH = '/content/drive/MyDrive/DL-Project/results/medium_task_final.csv'

os.makedirs(os.path.dirname(RESULTS_CSV_PATH), exist_ok=True)

# loading lyrics info
target_ids = np.load(os.path.join(SAVE_DIR, 'curated_lyrics_track_ids.npy'))
X_text_static = np.load(os.path.join(SAVE_DIR, 'text_embeddings.npy'))

# tuning grid
hp_grid = [
    # baseline
    {'n_mels': 64,  'resize_dim': 64,  'latent_dim': 16, 'n_epochs': 30},
    # for better pitch
    {'n_mels': 128, 'resize_dim': 64,  'latent_dim': 16, 'n_epochs': 30},
    # for capturing more textures
    {'n_mels': 64,  'resize_dim': 128, 'latent_dim': 16, 'n_epochs': 30},
]

#Audio extractions
def extract_audio_features(track_ids, n_mels, resize_dim):
    print(f"\n[Extraction] n_mels={n_mels}, size={resize_dim}x{resize_dim}")
    audio_images = []
    valid_ids = []

    for tid in tqdm(track_ids, desc="Processing Audio"):
        file_path = os.path.join(AUDIO_DIR, f"{tid}.wav")
        try:
            # 30s
            y, sr = librosa.load(file_path, sr=22050, duration=30)

            # Mel Spectrogram
            mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
            log_mels = librosa.power_to_db(mels, ref=np.max)

            # resizing
            img = cv2.resize(log_mels, (resize_dim, resize_dim), interpolation=cv2.INTER_AREA)

            # Min-Max Normalize
            img = (img - img.min()) / (img.max() - img.min() + 1e-6)

            audio_images.append(img[np.newaxis, :, :]) # (1, H, W)
            valid_ids.append(tid)
        except Exception as e:
            continue

    return np.array(audio_images), np.array(valid_ids)

# VAE
class VAE(nn.Module):
    def __init__(self, input_dim, text_dim=768, latent_dim=16):
        super(VAE, self).__init__()
        self.H, self.W = input_dim

        # Encoder, Standard CNN
        self.conv1 = nn.Conv2d(1, 32, 4, 2, 1)
        self.conv2 = nn.Conv2d(32, 64, 4, 2, 1)
        self.conv3 = nn.Conv2d(64, 128, 4, 2, 1)

        # calculating flatten dimentionas
        self.enc_h = self.H // 8
        self.enc_w = self.W // 8
        self.flatten_dim = 128 * self.enc_h * self.enc_w

        self.text_fc = nn.Sequential(nn.Linear(text_dim, 128), nn.ReLU())

        fusion_dim = self.flatten_dim + 128
        self.fc_mu = nn.Linear(fusion_dim, latent_dim)
        self.fc_logvar = nn.Linear(fusion_dim, latent_dim)

        # Decoder
        self.decoder_input = nn.Linear(latent_dim, self.flatten_dim)
        self.deconv1 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.deconv2 = nn.ConvTranspose2d(64, 32, 4, 2, 1)
        self.deconv3 = nn.ConvTranspose2d(32, 1, 4, 2, 1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, img, text):
        # Encode
        x = torch.relu(self.conv1(img))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(-1, self.flatten_dim)

        t = self.text_fc(text)
        h = torch.cat([x, t], dim=1)

        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)

        # Decode
        d = self.decoder_input(z)
        d = d.view(-1, 128, self.enc_h, self.enc_w)
        d = torch.relu(self.deconv1(d))
        d = torch.relu(self.deconv2(d))
        recon = torch.sigmoid(self.deconv3(d))
        return recon, mu, logvar

def train_vae_wrapper(X_audio, X_text, hp):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataset
    dataset = TensorDataset(torch.FloatTensor(X_audio).to(device), torch.FloatTensor(X_text).to(device))
    train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Model Init
    input_dim = (hp['resize_dim'], hp['resize_dim'])
    model = VAE(input_dim=input_dim, latent_dim=hp['latent_dim']).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Loss
    def loss_fn(recon_x, x, mu, logvar):
        BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return BCE + KLD

    # Training Loop
    model.train()
    for epoch in range(hp['n_epochs']):
        for img, txt in train_loader:
            optimizer.zero_grad()
            recon, mu, logvar = model(img, txt)
            loss = loss_fn(recon, img, mu, logvar)
            loss.backward()
            optimizer.step()

    # Extract Latent Z
    model.eval()
    z_list = []
    # no shuffling
    extract_loader = DataLoader(dataset, batch_size=32, shuffle=False)
    with torch.no_grad():
        for img, txt in extract_loader:
            _, mu, _ = model(img, txt)
            z_list.append(mu.cpu().numpy())

    return np.concatenate(z_list, axis=0)

# SCORES
def get_score(Z_features):
    # pptimizing for Silhouette Score
    kmeans = KMeans(n_clusters=8, random_state=42, n_init='auto')
    preds = kmeans.fit_predict(Z_features)
    if len(set(preds)) < 2: return -1
    return silhouette_score(Z_features, preds)

# -TUNING LOOPp
best_score = -1
best_config = None
best_Z = None
best_ids = None

print(f"Starting Hyperparameter Tuning on {len(hp_grid)} configs")

for i, hp in enumerate(hp_grid):
    print(f"Running Config {i+1}/{len(hp_grid)}: {hp}")

    X_audio_curr, valid_ids = extract_audio_features(target_ids, hp['n_mels'], hp['resize_dim'])

    mask = np.isin(target_ids, valid_ids)
    X_text_curr = X_text_static[mask]

    if len(X_audio_curr) == 0: continue

    Z_curr = train_vae_wrapper(X_audio_curr, X_text_curr, hp)

    score = get_score(Z_curr)
    print(f"Config Result -> Silhouette Score: {score:.4f}")

    # saving Best in Memory
    if score > best_score:
        print(f"New Best Founsdd! (Old: {best_score:.4f} vs neW: {score:.4f})")
        best_score = score
        best_config = hp
        best_Z = Z_curr
        best_ids = valid_ids

        # Save to Drive immediately
        np.save(os.path.join(SAVE_DIR, 'audio_images_64x64.npy'), X_audio_curr)
        np.save(os.path.join(SAVE_DIR, 'z_hybrid.npy'), Z_curr)
        np.save(os.path.join(SAVE_DIR, 'final_aligned_ids.npy'), valid_ids)


print(f"\n--- Generating Final Report for Best Config: {best_config} ---")

# Load Metadata for Labels
try:
    meta_df = pd.read_csv(META_PATH)
    meta_df['track_id'] = pd.to_numeric(meta_df['track_id'], errors='coerce')
    meta_df['primary_genre'] = meta_df['track_genre'].astype(str).str.split(',').str[0].str.strip()
    id_to_genre = dict(zip(meta_df['track_id'], meta_df['primary_genre']))
    true_labels = np.array([id_to_genre.get(tid, "Unknown") for tid in best_ids])
except Exception as e:
    print(f"Warning: Metadata not found or error ({e}). Genres will be 'Unknown'.")
    true_labels = np.array(["Unknown"] * len(best_ids))

# Generating PCA Baseline
print("Generating PCA Baseline for comparison...")
# Re-extracting text/audio for PCA alignment
X_audio_best = np.load(os.path.join(SAVE_DIR, 'audio_images_64x64.npy')).reshape(len(best_ids), -1)
mask_best = np.isin(target_ids, best_ids)
X_text_best = X_text_static[mask_best]
X_combined = np.concatenate([X_audio_best, X_text_best], axis=1)
X_combined_std = StandardScaler().fit_transform(X_combined)
Z_baseline = PCA(n_components=16).fit_transform(X_combined_std)

# VAE vs PCA
kmeans = KMeans(n_clusters=8, random_state=42)
vae_preds = kmeans.fit_predict(best_Z)
pca_preds = kmeans.fit_predict(Z_baseline)

# save csv
results_df = pd.DataFrame({
    'track_id': best_ids,
    'vae_cluster': vae_preds,
    'pca_cluster': pca_preds,
    'true_genre': true_labels
})
results_df.to_csv(RESULTS_CSV_PATH, index=False)
print(f"Final Analysis saved to: {RESULTS_CSV_PATH}")
print(f"Best Silhouette Score achieved: {best_score:.4f}")

Starting Hyperparameter Tuning on 3 configs
Running Config 1/3: {'n_mels': 64, 'resize_dim': 64, 'latent_dim': 16, 'n_epochs': 30}

[Extraction] n_mels=64, size=64x64


Processing Audio: 100%|██████████| 3216/3216 [25:42<00:00,  2.08it/s]


Config Result -> Silhouette Score: 0.0834
New Best Founsdd! (Old: -1.0000 vs neW: 0.0834)
Running Config 2/3: {'n_mels': 128, 'resize_dim': 64, 'latent_dim': 16, 'n_epochs': 30}

[Extraction] n_mels=128, size=64x64


Processing Audio: 100%|██████████| 3216/3216 [04:48<00:00, 11.14it/s]


Config Result -> Silhouette Score: 0.0853
New Best Founsdd! (Old: 0.0834 vs neW: 0.0853)
Running Config 3/3: {'n_mels': 64, 'resize_dim': 128, 'latent_dim': 16, 'n_epochs': 30}

[Extraction] n_mels=64, size=128x128


Processing Audio: 100%|██████████| 3216/3216 [04:23<00:00, 12.19it/s]


Config Result -> Silhouette Score: 0.0253

--- Generating Final Report for Best Config: {'n_mels': 128, 'resize_dim': 64, 'latent_dim': 16, 'n_epochs': 30} ---
Generating PCA Baseline for comparison...
Final Analysis saved to: /content/drive/MyDrive/DL-Project/results/medium_task_final.csv
Best Silhouette Score achieved: 0.0853


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

DATA_DIR = '/content/drive/MyDrive/DL-Project/data/processed_medium'
META_PATH = '/content/drive/MyDrive/DL-Project/metadata.csv'
RESULTS_CSV_PATH = '/content/drive/MyDrive/DL-Project/results/medium_task_final.csv'

try:
    best_Z = np.load(os.path.join(DATA_DIR, 'z_hybrid.npy'))
    best_ids = np.load(os.path.join(DATA_DIR, 'final_aligned_ids.npy'))
    print(f"Loaded features shape: {best_Z.shape}")
except FileNotFoundError:
    print("Error: Could not find 'z_hybrid.npy' ")
    raise

# loading GT fro ARI
print("Aligning Metadata")
try:
    meta_df = pd.read_csv(META_PATH)
    meta_df['track_id'] = pd.to_numeric(meta_df['track_id'], errors='coerce')
    meta_df['primary_genre'] = meta_df['track_genre'].astype(str).str.split(',').str[0].str.strip()
    id_to_genre = dict(zip(meta_df['track_id'], meta_df['primary_genre']))

    true_labels = np.array([id_to_genre.get(tid, "Unknown") for tid in best_ids])

    mask_ari = (true_labels != "Unknown") & (true_labels != "nan")
    print(f"Tracks with valid genres: {sum(mask_ari)} / {len(best_ids)}")
except Exception as e:
    print(f"Metadata Warning: {e}. ARI will be 0.")
    true_labels = np.array(["Unknown"] * len(best_ids))
    mask_ari = np.zeros(len(best_ids), dtype=bool)

#PCA Baseline
print("Generating PCA Baseline from raw data")
try:
    target_ids = np.load(os.path.join(DATA_DIR, 'curated_lyrics_track_ids.npy'))
    X_text_static = np.load(os.path.join(DATA_DIR, 'text_embeddings.npy'))

    X_audio_best = np.load(os.path.join(DATA_DIR, 'audio_images_64x64.npy'))
    X_audio_flat = X_audio_best.reshape(len(X_audio_best), -1)

    mask_text = np.isin(target_ids, best_ids)
    X_text_best = X_text_static[mask_text]

    # Checking shapes
    if len(X_audio_flat) != len(best_ids) or len(X_text_best) != len(best_ids):
        print("Warning: Raw feature shapes don't match VAE IDs exactly. Using Random Baseline instead of PCA.")
        Z_baseline = np.random.rand(len(best_ids), 16)
    else:
        # Concatenate & PCA
        X_combined = np.concatenate([X_audio_flat, X_text_best], axis=1)
        scaler = StandardScaler()
        X_combined_std = scaler.fit_transform(X_combined)
        pca = PCA(n_components=16)
        Z_baseline = pca.fit_transform(X_combined_std)

except Exception as e:
    print(f"Could not create PCA baseline ({e}). Skipping baseline comparison.")
    Z_baseline = None

# Evaluation Function
def evaluate_representation(data, name, labels_true, mask):
    if data is None: return {}
    print(f"Evaluating Representation: {name}")
    results = {}

    methods = {
        'KMeans': KMeans(n_clusters=8, random_state=42, n_init='auto'),
        'Agglomerative': AgglomerativeClustering(n_clusters=8),
        'DBSCAN': DBSCAN(eps=3.0, min_samples=5)
    }

    print(f"{'Method':<15} | {'Sil (High)':<10} | {'DBI (Low)':<10} | {'ARI (High)':<10} | Clusters")
    print("-" * 65)

    for method_name, model in methods.items():
        preds = model.fit_predict(data)
        n_clusters = len(set(preds))

        if n_clusters < 2:
            print(f"{method_name:<15} | {'FAILED':<10} | {'-':<10} | {'-':<10} | {n_clusters}")
            continue

        sil = silhouette_score(data, preds)
        dbi = davies_bouldin_score(data, preds)
        ari = adjusted_rand_score(labels_true[mask], preds[mask]) if sum(mask) > 0 else 0.0

        print(f"{method_name:<15} | {sil:<10.4f} | {dbi:<10.4f} | {ari:<10.4f} | {n_clusters}")
        results[f"{method_name}_preds"] = preds
    return results

# Run Comparison
results_vae = evaluate_representation(best_Z, "VAE (medium Model)", true_labels, mask_ari)
results_pca = evaluate_representation(Z_baseline, "PCA Baseline", true_labels, mask_ari)

# results
results_df = pd.DataFrame({
    'track_id': best_ids,
    'true_genre': true_labels,
    'vae_kmeans_cluster': results_vae.get('KMeans_preds', np.zeros(len(best_ids))),
    'pca_kmeans_cluster': results_pca.get('KMeans_preds', np.zeros(len(best_ids))),
})
results_df.to_csv(RESULTS_CSV_PATH, index=False)
print(f"Final Analysis saved to: {RESULTS_CSV_PATH}")

Loaded features shape: (3216, 16)
Aligning Metadata
Tracks with valid genres: 3216 / 3216
Generating PCA Baseline from raw data
Evaluating Representation: VAE (medium Model)
Method          | Sil (High) | DBI (Low)  | ARI (High) | Clusters
-----------------------------------------------------------------
KMeans          | 0.0853     | 1.9703     | 0.0753     | 8
Agglomerative   | 0.0545     | 2.3194     | 0.0625     | 8
DBSCAN          | 0.5098     | 3.3607     | 0.0001     | 2
Evaluating Representation: PCA Baseline
Method          | Sil (High) | DBI (Low)  | ARI (High) | Clusters
-----------------------------------------------------------------
KMeans          | 0.1191     | 1.8892     | 0.0470     | 8
Agglomerative   | 0.0685     | 2.3487     | 0.0397     | 8
DBSCAN          | FAILED     | -          | -          | 1
Final Analysis saved to: /content/drive/MyDrive/DL-Project/results/medium_task_final.csv
